In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time
import multiprocessing
import numpy as np
import pickle

In [ ]:
class TravianManager:
    def __init__(self, server):
        self.server = server
        self.init_driver()
        self.is_busy = False
    
    def init_driver(self):
        opts = Options()
        opts.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
        opts.add_argument("--start-maximized")
        opts.add_experimental_option("excludeSwitches", ['enable-automation']);
        self.driver = webdriver.Chrome(options=opts)
    
    def login(self, username, password):
        self.driver.get(self.server + '/login.php')
        time.sleep(2)
        self.driver.find_element_by_xpath('//input[@name="name"]').send_keys(username)
        self.driver.find_element_by_xpath('//input[@name="password"]').send_keys(password)
        self.driver.find_element_by_xpath('//input[@name="lowRes"]').click()
        self.driver.find_element_by_xpath('//button[@id="s1"]').click()
    
    def train(self, troop_type, count, every):
        self.train_thread = multiprocessing.Process(target=self.train_troop,args=(troop_type, count, every))
        self.train_thread.start()
    
    def train_troop(self, troop_type, count, every):
        while True:
            self.driver.get(self.server + '/build.php?gid=19')
            time.sleep(3)
            self.driver.find_element_by_xpath(f'//input[@class="text"][@name="t{troop_type}"]').send_keys(count)
            time.sleep(2)
            self.driver.find_element_by_xpath('//button[@name="s1"]').click()
            time.sleep(int(every *  60) + np.random.randint(2, 4))
    
    def start_farm(self, every=60):
        while True:
            self.driver.get(self.server + '/build.php?gid=16&tt=99')
            time.sleep(4)
            farms_list = self.driver.find_elements_by_xpath("//div[@class='raidListHeadline']")
            for farm in farms_list:
                farm.find_element_by_tag_name('form').find_element_by_tag_name('button').click()
                time.sleep(2 + np.random.randint(0, 3))
            time.sleep(int(60 * every))
    
    def square_around(self, x, y, fields):
        top_right = (x + fields, y + fields)
        bottom_left = (x - fields, y - fields)

        data = [top_right, bottom_left]

        for i in range(1, fields * 2 + 1):
            xi, yi = top_right
            data.append((xi - i, yi))
            data.append((xi, yi - i))

        for i in range(1, fields * 2):
            xi, yi = bottom_left
            data.append((xi + i, yi))
            data.append((xi, yi + i))

        # data = list(set(data))
        return data

    def area_around(self, x, y, fields):
        data = []
        for i in range(1, fields + 1):
            data.extend(self.square_around(x, y, i))
        return data

    def map_url(self, x, y):
        return self.server + '/position_details.php?x=%d&y=%d' % (x, y)
    
    def get_villages_urls_around(self, x, y, fields):
        return [(self.map_url(xi, yi), self.dst((x, y), (xi, yi))) for xi, yi in self.area_around(x, y, fields)]
    
    def dst(self, co1, co2):
        return round(((co2[0] - co1[0]) ** 2 + (co2[1] - co1[1]) ** 2) ** 0.5, 1)
    
    def get_all_players(self, max_pop):
        self.driver.get(self.server + '/statistics/player')
        
    def get_players(self, max_pops, save=True, load=True):
        if load:
            try:
                with open('players.pickle', 'rb') as handle:
                    players_list = pickle.load(handle)
                    return players_list
            except FileNotFoundError:
                pass
        self.driver.get(self.server + '/statistics/player')
        time.sleep(2)
        try:
            self.driver.find_element_by_xpath("//a[@class='last']").click()
        except:
            pass
        pages = int(tm.driver.current_url.split('=')[-1])
        players_list = []
        for page in range(pages, 0, -1):
            # get all players
            players_td = self.driver.find_elements_by_class_name("pla")
            pop_td = self.driver.find_elements_by_class_name("pop")
            players = zip(players_td, pop_td)
            for player_td, player_pops in players:
                pops = int(player_pops.text.strip())
                if pops <= max_pops:
                    ppp = {
                        'name': player_td.find_element_by_tag_name('a').text,
                        'pop': pops,
                        'url': player_td.find_element_by_tag_name('a').get_attribute('href')
                    }
                    players_list.append(ppp)
            try:
                if pops > max_pops:
                    break
            except UnboundLocalError:
                print('err23')
                pass
            time.sleep(1 + np.random.rand())
            self.driver.find_element_by_xpath("//a[@class='previous']").click()
        for p in players_list:
            self.get_player_villages(p)
        if(save):
            with open('players.pickle', 'wb') as handle:
                pickle.dump(players_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return players_list
    
    def get_player_villages(self, player):
        self.driver.get(player['url'])
        time.sleep(1)
        table = self.driver.find_element_by_xpath('//table[@id="villages"]')
        villages_td = table.find_element_by_tag_name('tbody').find_elements_by_class_name('name')
        # print(villages_td.source_code)
        villages = []
        for td in villages_td:
            vill = {
                'name': td.find_element_by_tag_name('a').text,
                'url': td.find_element_by_tag_name('a').get_attribute('href'),
            }
            print('Getting village:', vill, '...')
            self.driver.get(vill['url'])
            x_y_s = self.driver.current_url.split('?')[-1]
            vill['x'] = int(x_y_s.split('&')[0].split('=')[-1])
            vill['y'] = int(x_y_s.split('&')[1].split('=')[-1])
            villages.append(vill)
        player['villages'] = villages
        time.sleep(1 +  np.random.rand())
        
    def sort_villages(self, players, co):
        villages = []
        for p in players:
            villages.extend(p['villages'])
        for v in villages:
            v['dst'] = self.dst(co, (v['x'], v['y']))
        return sorted(villages, key=lambda x: x['dst'])
    
    def send_troops(self, troop_type, long_distance_troops, dst, village, count1=4, count2=2, x=0, y=0, long_dst_threshold=20):
        self.driver.get('https://norules.arabics.travian.com/build.php?gid=16&tt=2&newdid=' + village)
        time.sleep(3)
        ttype = troop_type
        troops_count = count1
        if dst > long_dst_threshold:
            ttype = long_distance_troops
            troops_count = count2
        self.driver.find_element_by_xpath('//input[@name="troops[0][%s]"]' % (ttype)).send_keys(troops_count)
        self.driver.find_element_by_xpath('//input[@name="x"]').send_keys(x)
        self.driver.find_element_by_xpath('//input[@name="y"]').send_keys(y)
        self.driver.find_element_by_xpath('//input[@name="c" and @value="4"]').click()
        self.driver.find_element_by_xpath('//button[@name="s1"]').click()
        time.sleep(1)
        to_send = int(self.driver.find_element_by_xpath('//input[@name="troops[0][%s]"]' % ttype).get_attribute('value'))
        if(to_send == troops_count):   
            time.sleep(1)
            try:
                error = self.driver.find_element_by_xpath('//p[@class="error"]')
                # print("didn't send attack to (%d|%d)" % (x, y))
                # print(error.text)
            except NoSuchElementException:
                pass
            self.driver.find_element_by_xpath('//button[@id="btn_ok"]').click()
            # print('Attack sent to (%d|%d)' % (x, y))
        else:
            print('Not enough')

In [ ]:
username = 'Test'
password = '123123'

# tm = TravianManager('https://norules.arabics.travian.com/')
tm = TravianManager('https://norules.arabics.travian.com')

In [ ]:
tm.login(username=username, password=password)

In [ ]:
player_list = tm.get_players(45, load=True)

In [ ]:
co = (9, -52)

In [ ]:
sorted_villages = tm.sort_villages(player_list, (-110, -28))

In [ ]:
# vill = sorted_villages[0]
# tm.send_troops('t3', 't5', dst=vill['dst'], count1=3, count2=2, x=vill['x'], y=vill['y'])

In [ ]:
def start_raid():
    def raid():
        while True:
            for co, village, fc, thresh in [[(9, -52), '3344', 120, 30], [(-110, -28), '70507', 30, 0]]:
                sorted_villages = tm.sort_villages(player_list, co)[:fc]
                tm.login(username=username, password=password)
                for (i, vill) in enumerate(sorted_villages):
                    print(f'[#{i}] sending to', vill)
                    try:
                        tm.send_troops('t3', 't5', village=village, dst=vill['dst'], count1=3, count2=2, x=vill['x'], y=vill['y'],
                                           long_dst_threshold=thresh)
                        time.sleep(2)
                    except Exception as e:
                        print(e)
            print('-------')
            print('sleeping')
            print('-------')
            time.sleep(0.35 * 60 * 60)
    process = multiprocessing.Process(target=raid)
    process.start()
    return process

In [ ]:
raid_process = start_raid()
# raid_process.terminate()

In [ ]:
# sorted_villages = tm.sort_villages(player_list, (-25, -23))

In [ ]:
# for vill in sorted_villages:
#     print("[x|y](%d|%d)[/x|y] -> fields: %.1f" % (vill['x'], vill['y'], vill['dst']))

In [ ]:
# outgoing: https://norules.arabics.travian.com/build.php?gid=16&tt=1&filter=2
# incomming: https://norules.arabics.travian.com/build.php?gid=16&tt=1&filter=1